In [ ]:
!pip install gradio transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 711.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F


In [ ]:
text = """Sending you sunshine and rainbows today! Just a reminder that you're awesome! Hope your day is filled with joy and laughter! The world needs more smiles, so let's spread some happiness! Enjoy the simple pleasures of life!
Thinking of you and wishing you a wonderful day!"""


emotions = predict_all_emotions(text)
print("Emotion Probabilities:")
for emotion, score in emotions:
    print(f"{emotion:<10} : {score:.4f}")

emotions = predict_all_emotions(text)
print("Emotion Probabilities:")
for emotion, score in emotions:
    print(f"{emotion:<10} : {score:.4f}")


NameError: name 'predict_all_emotions' is not defined

In [ ]:
# Load model and tokenizer
model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = model.config.id2label
def predict_emotions(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)[0]
    emotion_probs = {labels[i]: float(probs[i]) for i in range(len(probs))}
    return dict(sorted(emotion_probs.items(), key=lambda x: x[1], reverse=True))
gr.Interface(fn=predict_emotions,
             inputs="textbox",
             outputs="label",
             title="Emotion Detector",
             description="Enter text to get emotion probabilities."
).launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2aa1990e8579851b58.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Suggested responses per emotion
reply_suggestions = {
    "anger": "I understand you're upset. Do you want to talk more about what happened?",
    "disgust": "That sounds really unpleasant. I'm here if you want to share more.",
    "fear": "It's okay to feel scared sometimes. You're not alone.",
    "joy": "Aww, that’s so sweet! Sending a virtual hug 🤗",
    "neutral": "Thanks for sharing. Would you like to continue?",
    "sadness": "I'm sorry you're feeling this way. Want to talk about it?",
    "surprise": "Wow, that’s unexpected! What happened?"
}


def analyze_emotion(text):
    # Tokenize and predict
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)[0]

    # Map predictions to labels
    emotion_probs = {labels[i]: float(probs[i]) for i in range(len(probs))}
    sorted_emotions = sorted(emotion_probs.items(), key=lambda x: x[1], reverse=True)


    # Top emotion and suggested response
    top_emotion = sorted_emotions[0][0]
    suggested_reply = reply_suggestions.get(top_emotion, "Thank you for your message.")

    print(f"\nPredicted Emotion: {top_emotion} (Confidence: {sorted_emotions[0][1]:.2f})")
    print(f"Suggested Reply: {suggested_reply}")

# Example
text_input = input("Enter your message: ")
analyze_emotion(text_input)
